# EDA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns 
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import torch
import graphviz
from collections import OrderedDict

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
sns.set_theme(style="white")
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
PATH = ' '

## 1.0 Load data

* Selon la convention, il semble qu'il y ait des forecasts fait 2x par jour? À minuit et à midi? `2020123100 -> 31 decembre 2020 a minuit et 2020123112 -> 31 decembre 2020 a midi`?
    * Oui c'est ça.
    * En UTC
* Garder seulement les données des derniers mois de 2020 pour rentrer en mémoire

Add

```python
df['error_2t'] = df['gdps_2t'] - df['obs_2t']
df['squared_error_2t'] = (df['gdps_2t'] - df['obs_2t']) ** 2
df['step_hour'] = df['step'] / 3600
df['step_td'] = dd.to_timedelta(df['step'], unit='S')
df['valid'] = df['date'] + df['step_td']
```

In [ ]:
# df = pd.read_parquet(PATH + '/2019010100.parquet')
# df = pd.read_parquet(PATH)
df = pd.concat([pd.read_parquet(path) for path in glob.glob(PATH + '/*', recursive=True) if path.split('/')[-1][:5] == '20201'], ignore_index=True)

* La date, on assume que c'est à minuit? Et on ajoute le step (sec) pour avoir la bonne heure à partir de là?
    * UTC midi et minuit
* C'est quoi toutes les features `gdps`?
    * obs = vient des stations (pas le droit pour predire), c'est les mesures, on les a pas dans le fond au moment de la prediction/inference
    * gdps = vient du modele physique meteo, on peut prendre
* Les features `obs` c'est observé j'imagine? Ils veulent dire quoi sinon?
    * voir ci-haut
    
```python
['gdps_prate', qte de pluie/precipitation prevue
 'gdps_prmsl', pression, correlated avec altitude
 'gdps_2t', temperature a 2m
 'gdps_2d', point de rosee
 'gdps_2r', humidite relative
 'gdps_10u', composante de vent a 10m
 'gdps_10v', composante de vent a 10m
 'gdps_10si', vitesse m/s
 'gdps_10wdir', composante vent p/r au nord, ca peut flipper et fucker meme si cest presque pareil (359deg vs 1deg)
 'gdps_al', albedo (reflexivite de la surface de la terre)
 'gdps_t_850', temperature a la hauteur ou pression est 850 Pa
 'gdps_t_500', temperature a la hauteur ou pression est 500 Pa
 'gdps_gh_1000', altitude en m ou la pression est 1000 Pa
 'gdps_gh_850', altitude en m ou la pression est 850 Pa
 'gdps_gh_500', altitude en m ou la pression est 500 Pa
 'gdps_u_500', composante de vent ou la pression est 500 Pa
 'gdps_v_500', composante de vent ou la pression est 500 Pa
 'gdps_q_850', pression atmospherique ou la pression est 850 Pa
 'gdps_q_500', pression atmospherique ou la pression est 500 Pa
 'gdps_thick', difference de hauteur entre 2 couches]
```
 
* On cherche relation entre les variables: quand temperature haute eleve, on sous-estime la temperature
* Observation frequency change par station, verifier. Devrait etre 8 observations au moins par jour.
* Match previson et observation, par station. Donc normal que ce soit 8 observations.

In [ ]:
df.head()

In [ ]:
df.shape

## 2.0 Explore Data

* Comment on va splitter? Temporel par station (on prend pour chaque station toutes les données avant la date X pour train, le reste pour valid), temporel seulement (sans notion de station, on prend toutes les données avant la date X pour train, le reste pour valid), par station (X stations pour train, Y stations pour valid)
* Le target de notre modèle c'est l'erreur entre la variable prédite et celle observée en réalité? `df['gdps_2t'] - df['obs_2t']`?
    * 2t = 2m du sol, temperature de surface
    * On pourrait target vitesse du vent aussi

In [ ]:
df['error_2t'] = df['gdps_2t'] - df['obs_2t']

fig, axes = plt.subplots(nrows=2, ncols=1)
df[lambda x: x.station == 'CYBR']['error_2t'].plot(ax=axes[0])
axes[0].set_title('CYBR station')
df.groupby('step')['error_2t'].mean().plot(ax=axes[1])
axes[1].set_title('Mean of all stations');

* On dirait la shape de l'amérique du Nord. Sounds good. L'installation de geopandas marche pas pour une raison quelconque.

In [ ]:
df.groupby(['longitude','latitude']).size().reset_index().rename(columns={0:'count'}).plot.scatter(x='longitude', y='latitude', c='green')
plt.xlim(-200,0);

* La majorité des mesures proviennent du Canada.

In [ ]:
df.groupby(['longitude','latitude']).size().reset_index().rename(columns={0:'count'}).plot.scatter(x='longitude', y='latitude', c='count')
plt.xlim(-200,0);

* Mesures assez constantes dans le temps

In [ ]:
df.groupby('date').size().to_frame().rename(columns={0:'count'}).plot.line();

* Stations do not have the same timespan of data points
    * Check `2021-04-23-coverages.csv` pour avoir coverage (proportion, par station, du nombre de jours avec au moins 8 observations sur tous les jours)
    * Confirmer avec mes parquets que le coverage fonctionne

In [ ]:
timespan = df.groupby('station').agg({'date': [min, max]})

fig, ax = plt.subplots(figsize=(15,30))
ticklabels = timespan.index
ticks = np.arange(len(ticklabels))

ax.hlines(ticks, timespan['date']['min'], timespan['date']['max'], linewidth=1)
# ax.set_yticks(ticks)
# ax.set_yticklabels(ticklabels)
xfmt = mdates.DateFormatter('%Y/%m/%d\n%H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)
ax.set_xticks(pd.date_range(min(timespan['date']['min']), max(timespan['date']['max']), periods=5).tolist())
# ax.grid(axis='y')
ax.set_title('Time range for each station');

* The dataframe is already sorted. The next line has the same date or is a later date.

In [ ]:
((df.date - df.shift().date)).value_counts().compute()

## 3.0 Preparation

In [ ]:
df.head()

* Définition des features catégorielles/continues.
* Comment utiliser les variables obs?
* Utiliser station comme feature? Elevation? Step? Date? Lon/Lat? Le choix de les ignorer provient du notebook 027-dask-analysis. On veut être le plus général possible?
    * Modele generique ou par station (donc utiliser comme features)?
    * Un modele par station?
    * Fonction qui prend n'importe quel modele et essayer differents modeles sur chaque station.
    * Modele lineaire comme baseline+ (reg. lin. serait bon first step, compromis avec modele complexe)
    * Utiliser la date comme features c'est bon.
    * Sin wave pour encoder temporal features (voir notebook david)

In [ ]:
cat_columns = []
cont_columns = ['gdps_prate', 'gdps_prmsl', 'gdps_2t', 'gdps_2d', 'gdps_2r', 'gdps_10u', 'gdps_10v', 'gdps_10si', 
                'gdps_10wdir', 'gdps_al', 'gdps_t_850', 'gdps_t_500', 'gdps_gh_1000', 'gdps_gh_850', 'gdps_gh_500', 
                'gdps_u_500', 'gdps_v_500', 'gdps_q_850', 'gdps_q_500', 'gdps_thick']
target = 'error_2t'

* Split temporel naïf: on prendre X premières semaines pour train, X suivantes pour valid, X suivantes pour test
* 70/10/20
* shuffle=False pour garder l'ordre temporel

In [ ]:
X = df[cat_columns + cont_columns]
y = df[target]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, shuffle=False)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=2/3, shuffle=False)

assert abs(len(X_train) - 0.7*len(X)) < 1
assert abs(len(X_valid) - 0.1*len(X)) < 1
assert abs(len(X_test) - 0.2*len(X)) < 1
assert df.iloc[X_train.index[-1]].date <= df.iloc[X_valid.index[0]].date
assert df.iloc[X_valid.index[-1]].date <= df.iloc[X_test.index[0]].date

* Deal with missing data

In [ ]:
assert not X.isnull().values.any(), "There are NaN values in the dataframe"

* We do not find all stations that we trained on in the valid and test set

In [ ]:
train_stations = df.iloc[X_train.index].station.unique()
valid_stations = df.iloc[X_valid.index].station.unique()
test_stations = df.iloc[X_test.index].station.unique()
print(f"{len(set(train_stations) - set(valid_stations))} stations not in valid set")
print(f"{len(set(train_stations) - set(test_stations))} stations not in test set")

* DMatrix is an internal data structure that is used by XGBoost.

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)
dtest = xgb.DMatrix(X_test)

## 4.0 Training

### 4.0.0 Baseline

In [ ]:
baselines = OrderedDict()

* En moyenne, la prédiction est off de 3.13 degC par rapport à la valeur observée. La médiane est de 2.14 degC d'erreur.

In [ ]:
errors = df.iloc[X_train.index].error_2t
abs(errors).describe().to_frame()

* No error (error_2t=0) can be a valid baseline, without abs() the mean is basically zero.

In [ ]:
baselines['baseline_zero'] = 0
errors.describe().to_frame()

* La moyenne/médiane sur toutes les stations (train set) est un première baseline

In [ ]:
baselines['baseline_mean_pos'] = abs(errors).mean()
baselines['baseline_mean_neg'] = -abs(errors).mean()
baselines['baseline_median_pos'] = abs(errors).median()
baselines['baseline_median_neg'] = -abs(errors).median()

* La moyenne/médiane par station (train set) est un deuxième baseline

In [ ]:
baselines['baseline_station_mean_pos'] = abs(df.iloc[X_train.index].groupby('station').error_2t.mean())
baselines['baseline_station_mean_neg'] = -abs(df.iloc[X_train.index].groupby('station').error_2t.mean())
baselines['baseline_station_median_pos'] = abs(df.iloc[X_train.index].groupby('station').error_2t.median())
baselines['baseline_station_median_neg'] = -abs(df.iloc[X_train.index].groupby('station').error_2t.median())

* Prendre l'erreur de l'année précédente pour un tuple (station, date, step)

In [ ]:
# Not enough data analyzed for now because of memories issues

* Compute metrics for each baseline

In [ ]:
for k, v in baselines.items():
    if 'station' in k:
        predictions_ = df.iloc[X_test.index].merge(v, on='station', suffixes=('', '_pred')).error_2t_pred
    else:
        predictions_ = np.full(len(X_test), v)
    print(k)
    print(f'\tMAE: {mean_absolute_error(y_test, predictions_)}')
    print(f'\tRMSE: {mean_squared_error(y_test, predictions_, squared=False)}')

### 4.0.1 Train XGboost model

In [ ]:
num_boost_round = 100
params = {'objective': 'reg:squarederror', 
          'tree_method': 'gpu_hist' if torch.cuda.is_available() else 'hist',
          'eval_metric': ['rmse', 'mae']
         }
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
evals_result = {}

In [ ]:
model = xgb.train(params, dtrain, num_boost_round, evals=watchlist, verbose_eval=10, 
                  early_stopping_rounds=10, evals_result=evals_result)

* Why is RMSE loss lower for valid set?

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(evals_result['train']['rmse'], label='Train')
plt.plot(evals_result['valid']['rmse'], label='Valid')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('RMSE')
plt.title('RMSE Loss')
plt.show()

* J'aimerais analyser l'importance mais je ne sais pas ce que signifie chaque feature.

In [ ]:
xgb.plot_importance(model);

* J'aimerais analyser l'arbre mais je ne sais pas ce que signifie chaque feature.

In [ ]:
# Double click for bigger view
fig, ax = plt.subplots(figsize=(200,10))
xgb.plot_tree(model, ax=ax);

In [ ]:
predictions = model.predict(dtest)
print(f'MAE: {mean_absolute_error(y_test, predictions)}')
print(f'RMSE: {mean_squared_error(y_test, predictions, squared=False)}')

### 4.0.2 Train XGboost model, split temporal by dates

* Let's keep 2020-10-01 to 2020-12-01 as train set (2 months)
* 2020-12-02 to 2020-12-14 as valid set (2 weeks)
* 2020-12-15 to 2020-12-31 as test set (2 weeks)

In [ ]:
print(f'min: {df.date.min()}')
print(f'max: {df.date.max()}')

In [ ]:
train_idx = df[lambda x: x.date <= '2020-12-01'].index
valid_idx = df[lambda x: (x.date > '2020-12-01') & (x.date <= '2020-12-14')].index
test_idx = df[lambda x: x.date > '2020-12-14'].index

X_train = X.iloc[train_idx]
y_train = y.iloc[train_idx]
X_valid = X.iloc[valid_idx]
y_valid = y.iloc[valid_idx]
X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]

print(f'train: {len(X_train)/len(X)}, valid: {len(X_valid)/len(X)}, test: {len(X_test)/len(X)}')

* We do not find all stations that we trained on in the valid and test set

In [ ]:
train_stations = df.iloc[X_train.index].station.unique()
valid_stations = df.iloc[X_valid.index].station.unique()
test_stations = df.iloc[X_test.index].station.unique()
print(f"{len(set(train_stations) - set(valid_stations))} stations not in valid set")
print(f"{len(set(train_stations) - set(test_stations))} stations not in test set")

* DMatrix is an internal data structure that is used by XGBoost.

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)
dtest = xgb.DMatrix(X_test)

In [ ]:
num_boost_round = 100
params = {'objective': 'reg:squarederror', 
          'tree_method': 'gpu_hist' if torch.cuda.is_available() else 'hist',
          'eval_metric': ['rmse', 'mae'],
          'eta': 0.01         }
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
evals_result = {}

* With default like 4.0.1 we cannot converge.
* With lower lr (0.01) slightly better.
* Higher or lower max_depth does not do much.
* Difficult to converge

In [ ]:
model = xgb.train(params, dtrain, num_boost_round, evals=watchlist, verbose_eval=10, 
                  early_stopping_rounds=10, evals_result=evals_result)

* Now RMSE loss is not lower for valid set, which is normal, what is wrong in 4.0.1?

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(evals_result['train']['rmse'], label='Train')
plt.plot(evals_result['valid']['rmse'], label='Valid')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('RMSE')
plt.title('RMSE Loss')
plt.show()

* J'aimerais analyser l'importance mais je ne sais pas ce que signifie chaque feature.

In [ ]:
xgb.plot_importance(model);

* J'aimerais analyser l'arbre mais je ne sais pas ce que signifie chaque feature.

In [ ]:
# Double click for bigger view
fig, ax = plt.subplots(figsize=(200,10))
xgb.plot_tree(model, ax=ax);

In [ ]:
predictions = model.predict(dtest)
print(f'MAE: {mean_absolute_error(y_test, predictions)}')
print(f'RMSE: {mean_squared_error(y_test, predictions, squared=False)}')